<p style="font-size:36px"><b>Coursera Capstone Project</b><p>
<p style="font-size:20px">This notebook will contain the code for the capstone project of IBM data science course in Coursera.</p>

<p style="font-size:24px"><b>Week 3: Exercise 1</b><p>
<p>The next cells contain the creation of the initial dataframe with the different neighborhoods in Toronto.</p>

In [1]:
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup

In [2]:
toronto_table = []

# Download html page from Wikipedia

html = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M')
soup = BeautifulSoup(html.text, 'html.parser')

#Find the table with the desired data and create a dataframe, removing the empty cells in the process

table = soup.find('table')
for row in table.findAll('td'):
    neighborhoodRow = []
    neighborhoodRow.append(row.find('b').text)
    borough = row.find('span').text.split('(')[0]
    if borough != 'Not assigned':
        neighborhood = row.find('span').text.split('(')[1]
        neighborhood = neighborhood.replace(' /',',').strip(')').replace(')',' ')
        neighborhoodRow.append(borough)
        neighborhoodRow.append(neighborhood)
        toronto_table.append(neighborhoodRow)

toronto_df = pd.DataFrame(toronto_table, columns = ['PostalCode','Borough','Neighborhood'])

#Cleaning the cells with worng formatting

toronto_df['Borough']=toronto_df['Borough'].replace({'Downtown TorontoStn A PO Boxes25 The Esplanade':'Downtown Toronto Stn A',
                                                     'East TorontoBusiness reply mail Processing Centre969 Eastern':'East Toronto Business',
                                                     'EtobicokeNorthwest':'Etobicoke Northwest','East YorkEast Toronto':'East York/East Toronto',
                                                     'MississaugaCanada Post Gateway Processing Centre':'Mississauga'})

toronto_df.head(10)

,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Queen's Park,Ontario Provincial Government
5,M9A,Etobicoke,Islington Avenue
6,M1B,Scarborough,"Malvern, Rouge"
7,M3B,North York,Don Mills North
8,M4B,East York,"Parkview Hill, Woodbine Gardens"
9,M5B,Downtown Toronto,"Garden District, Ryerson"


<p style="font-size:24px"><b>Week 3: Exercise 2</b><p>
<p>The next cells contain the code to add the latitude and longitude of each neighborhood in Toronto using the package geocoder.</p>

In [ ]:
!pip install geocoder

In [ ]:
import geocoder 

toronto_df['Latitude'] = np.nan
toronto_df['Longitude'] = np.nan

for PC in toronto_df['PostalCode']:
    lat_lng_coords = None

    while(lat_lng_coords is None):
        g = geocoder.google('{}, Toronto, Ontario'.format(PC))
        lat_lng_coords = g.latlng

    toronto_df.loc[toronto_df['PostalCode']==PC,'Latitude'] = lat_lng_coords[0]
    toronto_df.loc[toronto_df['PostalCode']==PC,'Longitude'] = lat_lng_coords[1]

<p>Because the geocoder package didn't work, I will download the file provided by the course.</p>

In [3]:
!wget -O GeospatialCoordinates.csv https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DS0701EN-SkillsNetwork/labs_v1/Geospatial_Coordinates.csv

--2021-06-20 15:27:41--  https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DS0701EN-SkillsNetwork/labs_v1/Geospatial_Coordinates.csv
Resolving cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud (cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud)... 198.23.119.245
Connecting to cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud (cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud)|198.23.119.245|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2788 (2.7K) [text/csv]
Saving to: ‘GeospatialCoordinates.csv’

GeospatialCoordinat 100%[===================>]   2.72K  --.-KB/s    in 0s      

2021-06-20 15:27:42 (68.3 MB/s) - ‘GeospatialCoordinates.csv’ saved [2788/2788]



In [4]:
toronto_df['Latitude'] = np.nan
toronto_df['Longitude'] = np.nan

coord_df = pd.read_csv('GeospatialCoordinates.csv')

for PC in toronto_df['PostalCode']:
    toronto_df.loc[toronto_df['PostalCode']==PC,'Latitude'] = float(coord_df.loc[coord_df['Postal Code']==PC,'Latitude'])
    toronto_df.loc[toronto_df['PostalCode']==PC,'Longitude'] = float(coord_df.loc[coord_df['Postal Code']==PC,'Longitude'])

toronto_df.head(10)

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Queen's Park,Ontario Provincial Government,43.662301,-79.389494
5,M9A,Etobicoke,Islington Avenue,43.667856,-79.532242
6,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353
7,M3B,North York,Don Mills North,43.745906,-79.352188
8,M4B,East York,"Parkview Hill, Woodbine Gardens",43.706397,-79.309937
9,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937


<p style="font-size:24px"><b>Week 3: Exercise 3</b><p>
<p>The next cells contain the code to add the latitude and longitude of each neighborhood in Toronto using the package geocoder.</p>

In [6]:
!pip install folium

/opt/conda/envs/Python-3.7-main/lib/python3.7/site-packages/secretstorage/dhcrypto.py:16: CryptographyDeprecationWarning: int_from_bytes is deprecated, use int.from_bytes instead
  from cryptography.utils import int_from_bytes
/opt/conda/envs/Python-3.7-main/lib/python3.7/site-packages/secretstorage/util.py:25: CryptographyDeprecationWarning: int_from_bytes is deprecated, use int.from_bytes instead
  from cryptography.utils import int_from_bytes
     |████████████████████████████████| 94 kB 5.9 MB/s  eta 0:00:01


In [7]:
from geopy.geocoders import Nominatim
import folium

address = 'Toronto, Ontario'
geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)

toronto_subgroup_df = toronto_df.loc[toronto_df['Borough'].str.contains('Toronto')].reset_index(drop=True)
map_newyork = folium.Map(location=[location.latitude, location.longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, postal_code in zip(toronto_subgroup_df['Latitude'], toronto_subgroup_df['Longitude'], toronto_subgroup_df['Borough'], toronto_subgroup_df['PostalCode']):
    label = '{}, {}'.format(postal_code, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_newyork)  
    
map_newyork

In [49]:
# The code was removed by Watson Studio for sharing.

In [8]:

VERSION = '20180605' # Foursquare API version
LIMIT = 100 # A default Foursquare API limit value

def getNearbyVenues(postal_codes, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for postal_code, lat, lng in zip(postal_codes, latitudes, longitudes):           
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            postal_code, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['PostalCode', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [9]:
toronto_venues_df = getNearbyVenues(postal_codes=toronto_subgroup_df['PostalCode'],
                                      latitudes=toronto_subgroup_df['Latitude'],
                                      longitudes=toronto_subgroup_df['Longitude']
                                      )

In [10]:
toronto_onehot_df = pd.get_dummies(toronto_venues_df[['Venue Category']], prefix="", prefix_sep="")
toronto_onehot_df.insert(0,'PostalCode',toronto_venues_df['PostalCode']) 
toronto_grouped_df = toronto_onehot_df.groupby('PostalCode').mean().reset_index()
toronto_grouped_df

,PostalCode,Adult Boutique,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,...,Thrift / Vintage Store,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wine Shop,Yoga Studio
0,M4E,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.00000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,M4J,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.00000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2,M4K,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.024390,...,0.000000,0.024390,0.02439,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.024390
3,M4L,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.00000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
4,M4M,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.035714,...,0.035714,0.000000,0.00000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
5,M4N,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.00000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
6,M4P,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.00000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
7,M4R,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.00000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
8,M4S,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.038462,0.00000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
9,M4T,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.00000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


In [26]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]



In [28]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['PostalCode']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
venues_sorted = pd.DataFrame(columns=columns)
venues_sorted['PostalCode'] = toronto_grouped_df['PostalCode']

for ind in np.arange(toronto_grouped_df.shape[0]):
    venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped_df.iloc[ind, :], num_top_venues)

venues_sorted.head()

,PostalCode,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M4E,Health Food Store,Pub,Neighborhood,Yoga Studio,Dance Studio,Eastern European Restaurant,Donut Shop,Doner Restaurant,Dog Run,Distribution Center
1,M4J,Park,Metro Station,Convenience Store,Yoga Studio,Deli / Bodega,Eastern European Restaurant,Donut Shop,Doner Restaurant,Dog Run,Distribution Center
2,M4K,Greek Restaurant,Italian Restaurant,Coffee Shop,Ice Cream Shop,Yoga Studio,Bakery,Bank,Spa,Indian Restaurant,Dessert Shop
3,M4L,Fast Food Restaurant,Ice Cream Shop,Pub,Board Shop,Coffee Shop,Sandwich Place,Brewery,Restaurant,Burrito Place,Italian Restaurant
4,M4M,Coffee Shop,Gastropub,Café,Bakery,Comfort Food Restaurant,Ice Cream Shop,Latin American Restaurant,Seafood Restaurant,Brewery,Italian Restaurant


In [25]:
from sklearn.cluster import KMeans
from sklearn.model_selection import GridSearchCV

kmeans = KMeans()
parameters = {'n_clusters':range(2,10), 'random_state':[0]}
kmeans_search = GridSearchCV(kmeans, parameters)
kmeans_search.fit(toronto_grouped_df.drop('PostalCode', 1))
toronto_labels_df = kmeans_search.predict(toronto_grouped_df.drop('PostalCode', 1))


array([5, 4, 3, 3, 3, 4, 3, 3, 3, 1, 3, 4, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3,
       3, 2, 4, 3, 3, 3, 3, 3, 3, 0, 0, 3, 0, 0, 3, 3, 3], dtype=int32)

In [29]:
venues_sorted.insert(0, 'Cluster Labels', toronto_labels_df)

toronto_merged_df = toronto_subgroup_df

toronto_merged_df = toronto_merged_df.join(venues_sorted.set_index('PostalCode'), on='PostalCode')

toronto_merged_df.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636,3,Coffee Shop,Park,Pub,Bakery,Café,Performing Arts Venue,Breakfast Spot,Spa,Farmers Market,Beer Store
1,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937,3,Coffee Shop,Clothing Store,Sandwich Place,Café,Hotel,Thai Restaurant,Japanese Restaurant,Pizza Place,Bank,Cosmetics Shop
2,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418,3,Coffee Shop,Café,Italian Restaurant,Cocktail Bar,Restaurant,Clothing Store,Department Store,Gym,Farmers Market,Gastropub
3,M4E,East Toronto,The Beaches,43.676357,-79.293031,5,Health Food Store,Pub,Neighborhood,Yoga Studio,Dance Studio,Eastern European Restaurant,Donut Shop,Doner Restaurant,Dog Run,Distribution Center
4,M5E,Downtown Toronto,Berczy Park,43.644771,-79.373306,3,Cocktail Bar,Coffee Shop,Sandwich Place,Bakery,Vegetarian / Vegan Restaurant,Beer Bar,Farmers Market,Seafood Restaurant,Indian Restaurant,Japanese Restaurant


In [38]:
import matplotlib.cm as cm
import matplotlib.colors as colors

kclusters = kmeans_search.best_params_['n_clusters']
map_clusters = folium.Map(location=[location.latitude, location.longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged_df['Latitude'], toronto_merged_df['Longitude'], toronto_merged_df['PostalCode'], toronto_merged_df['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters